In [1]:
import numpy as np
import numba
from numba import jit , njit

In [2]:
# Global Config Variables
n0 = 1000  # number of p=0 points in metric space
V = n0  # Threshold for p=0
K = 10# No of clusters
A= 5# No of attributes
iterations = 1  # maximum iteration in clustering
runs =10
option='Kmeans'  #Kmedian
dataset ='Adult'  #Bank

In [3]:
import numpy as np
import os
import sys
import pandas
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import log2
from sklearn import preprocessing
import sys
import timeit


In [4]:
import datetime
import json
import random
from collections import defaultdict
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import cm
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import os



In [5]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.cluster import KMeans
!pip install scikit-learn-extra
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
from sklearn.metrics import pairwise_distances


In [ ]:
print(os.getcwd())
def load_Bank(data_dir=''):

    data_dir = data_dir
    _path = 'bank-full_p_6col.csv'
    data_path = os.path.join(data_dir, _path)

    K = 10

    df = pandas.read_csv(data_path, sep=',')
    #print(df.head())
    #print(len(df))
    
    return df


In [6]:
print(os.getcwd())
def load_Adult(data_dir=''):

    data_dir = data_dir
    _path = 'adult_p.csv'
    data_path = os.path.join(data_dir, _path)

    K = 10

    df = pandas.read_csv(data_path, sep=',')
    
    
    return df
load_Adult()

/home/hpc128/Downloads/Fresh Experiments-Again/Kmeans/Our RR


,age,fnlwgt,education_num,capital_gain,hours_per_week,gender
0,0.019622,-0.022667,-0.680470,0.725976,0.094976,0
1,0.296610,-0.787368,-0.357412,0.402068,-0.051703,0
2,-0.083487,-0.069366,0.479827,-0.822414,-0.285691,0
3,0.636832,-0.021345,0.256530,-0.721425,-0.087912,0
4,-0.393080,-0.017952,0.713519,0.574969,-0.073938,1
...,...,...,...,...,...,...
32556,-0.642387,-0.149353,0.484008,0.564430,-0.110399,1
32557,0.183154,-0.062405,-0.590824,-0.739884,-0.257021,0
32558,0.927796,-0.023090,-0.233823,-0.273761,-0.095099,1
32559,-0.577725,-0.786633,0.052733,-0.199630,-0.069348,0


In [7]:
if dataset=='Bank':
    df=load_Bank()
    df= df.round(decimals=5)
    print(len(df))
    df = df.dropna()
    print(len(df))
    df['type'] = df['type']-1
    typ = df['type'].values
    #print(len(typ))
    #print(df.head(10))
    c3 = np.count_nonzero(typ == 2)

    print(c1/(c1+c2+c3))
    print(c2/(c1+c2+c3))
    print(c3/(c1+c2+c3))


    print(c1)
    print(c2)
    print(c3)
    dfDropped = df.drop(columns=['type'])

elif dataset='Adult':
    df=load_Adult()
    df= df.round(decimals=5)
    print(len(df))
    df = df.dropna()
    print(len(df))
    #df['type'] = df['type']-1
    typ = df['gender'].values
    #print(len(typ))
    #print(df.head(10))
    c1 = np.count_nonzero(typ == 0)
    c2 = np.count_nonzero(typ == 1)

    print(c1/(c1+c2))
    print(c2/(c1+c2))

    print(c1)
    print(c2)
    dfDropped = df.drop(columns=['gender'])

32561
32561
0.6692054912318418
0.33079450876815825
21790
10771


In [8]:
def VanillaKmeans(X, k,seedValue):
    """
    specify the number of clusters k and
    the maximum iteration to run the algorithm
    """
    n_row, n_col = X.shape
    maxiter=100
    
    # randomly choose k data points as initial centroids
   
    
    rand_indices = np.random.choice(n_row, size = k)
    centroids = X[rand_indices]
    cost_variation=[]
    cnt =0
    for itr in range(maxiter):
        # compute distances between each data point and the set of centroids
        # and assign each data point to the closest centroid
        distances_to_centroids = pairwise_distances(X, centroids, metric = 'sqeuclidean',force_all_finite=True)
        cluster_assignment = np.argmin(distances_to_centroids, axis = 1)

        # select all data points that belong to cluster i and compute
        # the mean of these data points (each feature individually)
        # this will be our new cluster centroids
        new_centroids=[]
        for i  in range(k):
            #print(i)
            assign= np.array([X[cluster_assignment == i]])
          
            med = np.mean(assign[0],axis=0)
            new_centroids.append(med)             
        
        
        # if the updated centroid is still the same,
        # then the algorithm converged
        new_centroids = np.array(new_centroids)
        #print("Median"+str(new_centroids))
            
        #print("Centrer"+str(centroids))
        if np.all(centroids == new_centroids):
            cnt = cnt+1
            if cnt ==4:
                break
        
        centroids = new_centroids
        
        heterogeneity = 0
        for i in range(k):
            # note that pairwise_distance only accepts 2d-array
            cluster_data = X[cluster_assignment == i]
            distances = pairwise_distances(cluster_data, [centroids[i]], metric = 'euclidean')
            heterogeneity += np.sum(distances ** 2) #sq euclidean 
        cost_variation.append(heterogeneity)
        
    
    return centroids, cluster_assignment, heterogeneity,cost_variation#(cost)


def VanillaKmedian(X, k,seedValue):
    """
    specify the number of clusters k and
    the maximum iteration to run the algorithm
    """
    n_row, n_col = X.shape
    maxiter=100
    
    # randomly choose k data points as initial centroids
    #if seed is not None:
     #   np.random.seed(seed)
    
    rand_indices = np.random.choice(n_row, size = k)
    centroids = X[rand_indices]
    cost_variation=[]
    cnt =0
    for itr in range(maxiter):
        # compute distances between each data point and the set of centroids
        # and assign each data point to the closest centroid
        distances_to_centroids = pairwise_distances(X, centroids, metric = 'sqeuclidean',force_all_finite=True)
        cluster_assignment = np.argmin(distances_to_centroids, axis = 1)

        # select all data points that belong to cluster i and compute
        # the mean of these data points (each feature individually)
        # this will be our new cluster centroids
        new_centroids=[]
        for i  in range(k):
            #print(i)
            assign= np.array([X[cluster_assignment == i]])
            med = np.median(assign[0],axis=0)
            new_centroids.append(med)             
       
        # if the updated centroid is still the same,
        # then the algorithm converged
        new_centroids = np.array(new_centroids)
       
        if np.all(centroids == new_centroids):
    
                break
        
        centroids = new_centroids
        
        heterogeneity = 0
        for i in range(k):
            # note that pairwise_distance only accepts 2d-array
            cluster_data = X[cluster_assignment == i]
            distances = pairwise_distances(cluster_data, [centroids[i]], metric = 'euclidean')
            heterogeneity += np.sum(distances ** 2) #sq euclidean 
        cost_variation.append(heterogeneity)
        
    
    return centroids, cluster_assignment, heterogeneity,cost_variation#(cost)


def VanillaKmedian(X, k,seedValue):
    """
    specify the number of clusters k and
    the maximum iteration to run the algorithm
    """
    n_row, n_col = X.shape
    maxiter=100
    
    # randomly choose k data points as initial centroids
    #if seed is not None:
     #   np.random.seed(seed)
    
    rand_indices = np.random.choice(n_row, size = k)
    centroids = X[rand_indices]
    cost_variation=[]
    cnt =0
    for itr in range(maxiter):
        # compute distances between each data point and the set of centroids
        # and assign each data point to the closest centroid
        distances_to_centroids = pairwise_distances(X, centroids, metric = 'sqeuclidean',force_all_finite=True)
        cluster_assignment = np.argmin(distances_to_centroids, axis = 1)

        # select all data points that belong to cluster i and compute
        # the mean of these data points (each feature individually)
        # this will be our new cluster centroids
        new_centroids=[]
        for i  in range(k):
            #print(i)
            assign= np.array([X[cluster_assignment == i]])
            med = np.median(assign[0],axis=0)
            new_centroids.append(med)             
       
        # if the updated centroid is still the same,
        # then the algorithm converged
        new_centroids = np.array(new_centroids)
       
        if np.all(centroids == new_centroids):
    
                break
        
        centroids = new_centroids
        
        heterogeneity = 0
        for i in range(k):
            # note that pairwise_distance only accepts 2d-array
            cluster_data = X[cluster_assignment == i]
            distances = pairwise_distances(cluster_data, [centroids[i]], metric = 'euclidean')
            heterogeneity += np.sum(distances ** 2) #sq euclidean 
        cost_variation.append(heterogeneity)
        
    
    return centroids, cluster_assignment, heterogeneity,cost_variation#(cost)

In [9]:
def dual_print(f,*args,**kwargs):
    #print(*args,**kwargs)
    print(*args,**kwargs,file=f)

def load_dataset(csv_name):
    # read the dataset from csv_name and return as pandas dataframe
    df = pd.read_csv(csv_name, header=None)
    return df


def k_random_index(df,K):
    # return k random indexes in range of dataframe
    return random.sample(range(0, len(df)), K)

def find_k_initial_centroid_Kmeans(df,K,seedValue):
    kmeans=KMeans(n_clusters=int(K),random_state=int(seedValue)).fit(df)
    return kmeans.cluster_centers_

          

def find_k_initial_centroid(df,K):
    centroids = []    # make of form [ [x1,y1]....]

    rnd_idx = k_random_index(df,K)
    
    for i in rnd_idx:
        coordinates =[]
        for a in range(0,A):
            coordinates.append(df.loc[i][a])
        centroids.append(coordinates)   #df is X,Y,....., Type

    return centroids

#nOt using
def calc_distance(x1, y1, x2, y2):
    # returns the euclidean distance between two points
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def calc_distance_a(centroid, point):
    #print('çalculating distance\n')

    sum_ = 0

    for i in range(0, len(centroid)):
        sum_ = sum_ + (centroid[i]-point[i])**2

    return sum_ #**0.5

@njit(parallel=False)
def find_distances_fast(k_centroids, df):
    #print("Inside fast distances")

   
    dist = np.zeros((len(k_centroids),len(df),A+2),np.float64)
    Kcnt = 0 
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
        #print("C: "+str(Kcnt))
        l = np.zeros((len(df),A+2),np.float64)
        
       # for row in df:
        index = 0 
        for row in df:                # row is now x,y,z......type
            # append all coordinates to point
            dis = np.sum((c- row[:A])**2)#calc_distance_a(c, point)
            #Processing the vector for list
            row_list = np.array([dis])
            #append distance or l norm
            row_list = np.append(row_list,row[:A+1])
            #append all coordinates #append type of this row
  
            l[index] = row_list
            index = index + 1
            #l.append([calc_distance(c[0], c[1], row[0], row[1]), row[0], row[1], row[2]])  # [dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist[Kcnt]= l
        Kcnt = Kcnt + 1

    # return dist which contains distances of all points from every centroid

    return dist

def find_distances(k_centroids, df):
    dist = []
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
        l = []
      

        for index, row in df.iterrows():                # row is now x,y,z......type
            point =[]
            for a in range(0, A):
                point.append(row.iloc[a])  # append all coordinates

            dis = calc_distance_a(c, point)
            #Processing the vector for list
            row_list = [dis]
            #append distance or l norm

            for a in range(0, A):
                row_list.append(row.iloc[a])   #append all coordinates
            #print(row.iloc[a+1])
            row_list.append(row.iloc[a+1])   #append type of this row

            l.append(row_list)
            #l.append([calc_distance(c[0], c[1], row[0], row[1]), row[0], row[1], row[2]])  # [dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist.append(l)

    # return dist which contains distances of all points from every centroid

    return dist


def sort_and_valuation(dist):
    sorted_val = []
  
    for each_centroid_list in dist:
        each_centroid_list_sorted = sorted(each_centroid_list, key=lambda x: (x[A+1], x[0]))  # A+1 is index of type , 0 is dist
        sorted_val.append(each_centroid_list_sorted)

        # sort on basis of type & then dist.
        # Now all whites are towards start and all black are after white as they have additional V added to their valuation
        # Among the whites, the most closest is at start of list as it has more valuation.
        # Similarly sort the black points among them based on distance as did with white

    return sorted_val


def clustering(sorted_valuation, hashmap_points,K):
    n = len(hashmap_points.keys())  # total number of points in metric space
    
    
    cluster_assign = []

    for i in range(0, K):
        cluster_assign.append([])  # initially all clusters are empty
  
    map_index_cluster = []
    for i in range(0,K+2):
        map_index_cluster.append(0)
        #initially check all sorted evaluation from 0th index 
    
    number_of_point_alloc = 0
    curr_cluster = 0
    
    # until all points are allocated
    while number_of_point_alloc != n:  # As convergence is guaranteed that all points will be allocated to some cluster set
        #print('Number of point alloc : '+str(number_of_point_alloc))
        start_inde = map_index_cluster[curr_cluster % K]
        
        for inde in range(start_inde,len(sorted_valuation[curr_cluster % K])):
            each = sorted_valuation[curr_cluster % K][inde]
            # each is (dist,X,Y,....Z,type)
          
            
            if hashmap_points[tuple(each[1: -1])] == 0:    # each is (dist, X,Y,....Z, type)
                cluster_assign[curr_cluster].append(each)
                hashmap_points[tuple(each[1: -1])] = 1
                number_of_point_alloc += 1
                map_index_cluster[curr_cluster % K] = inde  #next time start from here as isse prev all allocated
                break

        curr_cluster = (curr_cluster + 1) % K

    return cluster_assign


def update_centroids_median(cluster_assign,K):
    new_centroids = []
    for k in range(0, K):
      
        cAk =  np.array(cluster_assign[k])
        cAk = np.delete(cAk,[0,-1],axis=1)
        #print(len(cAk))
        if len(cAk) %2 ==0 and len(cAk)>0: 
            cc = [np.median(np.array(cAk[:-1])[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk) %2 !=0 and len(cAk)>0:
            cc = [np.median(np.array(cAk)[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk)==0:
            print("Error: No centroid found updation error")
  
    return new_centroids
        


def update_centroids(cluster_assign,K):

    new_centroids = []
    for k in range(0, K):

        sum_a = []

        for i in range(0, A):
            sum_a.append(0)

        for each in cluster_assign[k]:
            sum_a = [sum(x) for x in zip(sum_a, each[1:-1])]
            #each is (dist,X,Y,.....Z,type)

        new_coordinates = []
        for a in range(0, A):
            new_coordinates.append(sum_a[a] / len(cluster_assign[k]))
        new_centroids.append(new_coordinates)
        k=k+1



    return new_centroids


def calc_clustering_objective(k_centroid, cluster_assign,K):
    cost = 0

    for k in range(0, K):

        for each in cluster_assign[k]:  #each is (dist, X,Y,....,Z,type)
            dd = calc_distance_a(k_centroid[k], each[1:-1])
            cost = cost + (dd)

    return cost


def calc_fairness_error(df, cluster_assign,K):
    U = []  # distribution of each type in original target dataset for each J = 0 , 1....
    P_k_sum_over_j = []  # distribution in kth cluster  sum_k( sum_j(   Uj * j wale/total_in_cluster ) )

    f_error = 0
    cnt_j_0 = 0
    cnt_j_1 = 0
  #  cnt_j_2 = 0
    cnt = 0
    for index, row in df.iterrows():
        if row.iloc[-1] == 1:
            cnt_j_1 += 1
        elif row.iloc[-1] == 0:
            cnt_j_0 += 1
      #  elif row.iloc[-1] == 2:
        #    cnt_j_2 += 1
            
        cnt += 1

    U.append(cnt_j_0 / cnt)
    U.append(cnt_j_1 / cnt)
    #U.append(cnt_j_2 / cnt)

    

    for k in range(0, K):  # for each cluster

        for j in range(0, len(U)):   #for each demographic group

            cnt_j_cluster = 0
            cnt_total = 0

            for each in cluster_assign[k]:
                if int(each[-1]) == j:    #each is (dist,X, Y.....,Z,type)
                    cnt_j_cluster += 1
                cnt_total += 1
                
            if cnt_j_cluster !=0 and cnt_total != 0:
                P_k_sum_over_j.append(-U[j] * np.log((cnt_j_cluster / cnt_total)/U[j]))
            else:
                P_k_sum_over_j.append(0)  #log(0)=0 considered

    for each in P_k_sum_over_j:
        f_error += each

    return f_error


def calc_balance(cluster_assign,K):
    S_k = []  # balance of each k cluster
    balance = 0  # min (S_k)

    for k in range(0, K):
        cnt_j_0 = 0
        cnt_j_1 = 0
       # cnt_j_2 = 0
        cnt = 0
        for each in cluster_assign[k]:

            if int(each[-1]) == 1:
                cnt_j_1 += 1
            elif int(each[-1]) == 0:
                cnt_j_0 += 1
           # elif int(each[-1]) == 2:
           #     cnt_j_2 += 1
                
            cnt += 1

        if cnt_j_0 != 0 and cnt_j_1 != 0 :#and cnt_j_2!= 0:
            S_k.append(min([cnt_j_0 / cnt_j_1, cnt_j_1 / cnt_j_0 ]))#, cnt_j_1 / cnt_j_2 , cnt_j_2 / cnt_j_1 , cnt_j_0 / cnt_j_2, cnt_j_2 / cnt_j_0 ]))
        elif cnt_j_0 == 0 or cnt_j_1 ==0  :#or cnt_j_2==0:
            S_k.append(0)



    balance = min(S_k)

    return balance


In [15]:


def main():
    # Step1 : Load the dataset
            
    list_fair_K=[]
    list_obj_K =[]       
    list_balance_K=[]
    
    os.makedirs('Adult_kmeans_center')
    
    for kk in [2,5,10,15,20,30,40]:
        K = kk
        
        print(" K=="+str(K)+"  ")
        
        list_fair_run=[]
        cost_variation_ck_collec =[]
        cost_variation_collec = []
        list_obj_run =[]       
        list_balance_run=[]
        seeds = [0,100,200,300,400,500,600,700,800,900,1000,1100]
      
        for run in range(0,runs):
            np.random.seed(seeds[run])
            random.seed(seeds[run])
            f = open('Adult_kmeans_center/K_'+str(K)+'_run_'+str(run)+'_output.txt', 'a')
        

            list_fair_iter=[]
            list_obj_iter =[]
            list_balance_iter=[]

            # Step2 : Find initial K random centroids using k_random_index(df) & find_k_initial_centroid(df)
            if option=='Kmeans':
                k_centroid,_,_,cost_variation= VanillaKmeans(dfDropped.values,kk,seeds[run])#VanillaKmeans()find_k_initial_centroid_Kmeans(dfDropped,kk,seeds[run])#find_k_initial_centroid(df,kk)
            else:
                k_centroid,_,_,cost_variation= VanillaKmedian(dfDropped.values,kk,seeds[run])#VanillaKmeans()find_k_initial_centroid_Kmeans(dfDropped,kk,seeds[run])#find_k_initial_centroid(df,kk)
            cost_variation_ck = []
            
            for eac in cost_variation:
                cost_variation_ck.append(eac)
            
           
            prev_assignment =[]
            cluster_assignment = []

            for i in range(0, K):
                cluster_assignment.append([])  # initially all clusters are empty

            sum_time = 0
            curr_itr = 0
            prev_objective_cost=-1
            objective_cost = 0
                # Step3 : Find distances from the centroids using find_distances() with list of [ [x1,y1,z1..] , [x2,y2,z2..]....] centroids format list
            while curr_itr < iterations:# and prev_objective_cost != objective_cost:

                start = time.process_time()#timeit.default_timer()
             
                df1 = df.values
                k_centroids1= np.array(k_centroid)
                
                dist = find_distances_fast(k_centroids1, df1)
             
                valuation = sort_and_valuation(dist)
               
                #Step5 : Perform clustering using valuation matrix & hashmap of all points in metric
                hash_map = {}
                for index, row in df.iterrows():
                    temp = tuple(row[:-1])
                    hash_map.update({tuple(row[:-1]): 0})   #dict is of form { (x,y): 0 , ....}
            
                prev_assignment = cluster_assignment
                cluster_assignment = clustering(valuation, hash_map,K)

                
                balance = calc_balance(cluster_assignment,K)
              
                f_error = calc_fairness_error(df, cluster_assignment,K)
               
                clustering_cost = calc_clustering_objective(k_centroid,cluster_assignment,K)
                if curr_itr!=0:
                     prev_objective_cost = objective_cost
                    
                objective_cost = np.round(clustering_cost,3)
                
                cost_variation_ck.append(objective_cost)
                

                list_balance_iter.append(str(balance))
                list_obj_iter.append(str(objective_cost))
                list_fair_iter.append(str(f_error))

              

              

                #Step7 : Find new centroids using mean of all points in current assignment
                if option=='Kmeans':
                    k_centroid_temp = update_centroids(cluster_assignment,K)
                else:
                    k_centroid_temp = update_centroids_median(cluster_assignment,K)
                
                clustering_cost_temp = calc_clustering_objective(k_centroid_temp,cluster_assignment,K)
                cost_variation.append(np.round(clustering_cost_temp,3))
               
                stop = time.process_time()#timeit.default_timer()
                sum_time += (stop - start)
               
                curr_itr += 1

                dual_print(f,'-----------------------------Finished-----------------------------------------------\n')


              
           
            #Step 10 : Find balance , fairness error , and clustering objective or cost

            balance_converged = calc_balance(cluster_assignment,K)
            f_error_converged = calc_fairness_error(df, cluster_assignment,K)
            clustering_cost_converged = calc_clustering_objective(k_centroid,cluster_assignment,K)

            #
            cost_variation_collec.append(cost_variation)
          
            #cost_variation_ck_collec.append(cost_variation_ck)
            
            f.close()
            run  = run +1
            list_obj_run.append(clustering_cost_converged)
            list_fair_run.append(f_error_converged)
            list_balance_run.append(balance_converged)
        
        
        print("Cost variations over run\n")
        print(str(cost_variation_collec))
        print("\nCost variations over run- CK\n")
        print(str(cost_variation_ck_collec))
        print("\nbalance variations over run")
        print(str(list_balance_run))
        print("\nfairness error over run")
        print(str(list_fair_run))
        print("#"*30)
        
        
        list_obj_K.append(np.mean(np.array(list_obj_run)))
        list_fair_K.append(np.mean(np.array(list_fair_run)))
        list_balance_K.append(np.mean(np.array(list_balance_run)))
        
    

In [16]:

import time
if __name__ == "__main__":
    main()
    

 K==2  
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                        RUN  : 0
[25834.781121634544, 25356.457852122825, 25106.651447890734, 24983.762469445952, 24926.752043846114, 24899.117271444076, 24885.26467136869, 24877.5632594853, 24872.61128980082, 24869.838000597087, 24868.344990783746, 24867.452725855306, 24866.927998748433, 24866.545949184547, 24866.367600362202, 24866.281424170804, 24866.185173726266, 24866.10489410742, 24866.034842417408, 24866.01621196475, 24866.01621196475, 24866.01621196475, 24866.01621196475]
Total time taken to converge 10.391921649000324

Iterations total taken for convergence : 0


Cost variation over iterations
['24915.183']

Balance variation over iterations
['0.49615776317007687']

Fairness error over iterations
['3.2434077390586583e-06']


Final converged balance : 0.49615776317007687

Final Converged Fairness Error : 3.2434077390586583e-06

Final converged Clustering Objective/Cost 2

[25951.32763153725, 25682.910714784448, 25432.88337200765, 25239.478868195314, 25106.597657434282, 25019.60568697812, 24966.46108513412, 24934.50974939168, 24911.032664974635, 24897.349903699687, 24887.2431251968, 24880.436327301693, 24876.135518983563, 24872.89672950079, 24870.571928904756, 24869.159183577533, 24868.13003880414, 24867.568629442256, 24867.084756914075, 24866.710388138, 24866.511276519115, 24866.42603571479, 24866.369071331363, 24866.336579039416, 24866.31481996685, 24866.28483526418, 24866.26346795445, 24866.244012445706, 24866.229258190036, 24866.211381613866, 24866.191276401347, 24866.152459132507, 24866.092290215987, 24866.04775813962, 24866.024461159846, 24866.018643126394, 24866.018429441996, 24866.018429441996, 24866.018429441996, 24866.018429441996]
Total time taken to converge 9.321623584000008

Iterations total taken for convergence : 0


Cost variation over iterations
['24915.135']

Balance variation over iterations
['0.49615776317007687']

Fairness error ove


Cost variation over iterations
['15786.842']

Balance variation over iterations
['0.4959500115713955']

Fairness error over iterations
['8.144449203607114e-06']


Final converged balance : 0.4959500115713955

Final Converged Fairness Error : 8.144449203607114e-06

Final converged Clustering Objective/Cost 15786.842037738505

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                        RUN  : 3
[17548.52819006283, 16274.778790284669, 15346.886002649368, 14947.273964963511, 14784.534727282395, 14694.079621692563, 14647.791324690592, 14622.884714954786, 14607.090631449813, 14595.692807534773, 14587.323044951667, 14580.327469838718, 14574.460218476213, 14570.108927461652, 14566.927093841234, 14564.247968992757, 14562.306053127912, 14560.957043027282, 14559.950183420216, 14559.24317557115, 14558.645008623287, 14558.011331358779, 14557.383325272578, 14556.837829423812, 14556.400920010656, 14555.925110382457, 14555.376611869124,

[19450.36734275124, 17546.574359571456, 16148.951817092246, 15380.98665960138, 15063.952261932893, 14844.757471934048, 14663.10393635225, 14547.075805233117, 14486.225953354106, 14452.352964160002, 14430.688355166645, 14424.296909822831, 14421.95065257862, 14421.030465791358, 14420.54475464582, 14420.286436211798, 14420.12568252695, 14419.968560201827, 14419.872695311602, 14419.825597152485, 14419.798037466957, 14419.780254670293, 14419.77167469067, 14419.761211540643, 14419.760242676335, 14419.75848940274, 14419.758132363728, 14419.758132363728, 14419.758132363728, 14419.758132363728]
Total time taken to converge 11.045477362999918

Iterations total taken for convergence : 0


Cost variation over iterations
['15786.89']

Balance variation over iterations
['0.4959500115713955']

Fairness error over iterations
['8.144449203607114e-06']


Final converged balance : 0.4959500115713955

Final Converged Fairness Error : 8.144449203607114e-06

Final converged Clustering Objective/Cost 15786.8

Total time taken to converge 12.279203242999756

Iterations total taken for convergence : 0


Cost variation over iterations
['12481.952']

Balance variation over iterations
['0.4956038870893105']

Fairness error over iterations
['1.640868668946482e-05']


Final converged balance : 0.4956038870893105

Final Converged Fairness Error : 1.640868668946482e-05

Final converged Clustering Objective/Cost 12481.95193956794

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                        RUN  : 3
[13259.889338075627, 11215.183560980557, 10373.331346181076, 9938.529697250897, 9744.547768860564, 9663.05099000827, 9617.839112545405, 9591.236078941341, 9571.380982913488, 9553.638429595201, 9537.709383977959, 9525.610073709591, 9517.918478459347, 9513.648722604376, 9511.058672835457, 9509.599291281747, 9508.69036711375, 9508.021069153721, 9507.72799740987, 9507.587946067375, 9507.501614672568, 9507.41806394335, 9507.32481368958, 9507.26693

...

Total time taken to converge 13.773482478000005

Iterations total taken for convergence : 0


Cost variation over iterations
['7287.841']

Balance variation over iterations
['0.49514563106796117']

Fairness error over iterations
['5.4981463993622774e-05']


Final converged balance : 0.49514563106796117

Final Converged Fairness Error : 5.4981463993622774e-05

Final converged Clustering Objective/Cost 7287.841330342306

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                        RUN  : 3
[6136.371583526897, 5343.285696206493, 5122.322934667658, 5035.868440690902, 4992.04161958456, 4963.706992629502, 4945.296540478538, 4932.3174428633065, 4922.873975216441, 4915.936273149253, 4908.218579168853, 4897.904477503199, 4886.559270232758, 4874.594023618481, 4865.454411458349, 4858.523212628278, 4853.271232010013, 4850.202169610887, 4848.357355880874, 4847.239450171425, 4846.380676634894, 4845.878657596724, 4845.3930301207865, 4844

Total time taken to converge 13.659792859001755

Iterations total taken for convergence : 0


Cost variation over iterations
['7455.038']

Balance variation over iterations
['0.49514563106796117']

Fairness error over iterations
['5.4981463993622774e-05']


Final converged balance : 0.49514563106796117

Final Converged Fairness Error : 5.4981463993622774e-05

Final converged Clustering Objective/Cost 7455.038321288611

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                        RUN  : 7
[6200.122842529543, 5440.456481006863, 5262.766265654591, 5175.283315208714, 5121.977816455823, 5083.76490841504, 5050.262383585205, 5026.705330329687, 5005.947986934594, 4982.8446204241945, 4956.753669171449, 4922.290972355717, 4874.482656816734, 4821.826300137432, 4786.287569702108, 4765.106900706431, 4753.575692507732, 4745.224643013326, 4739.408270245932, 4734.887246907612, 4731.203203159631, 4728.0250064514385, 4725.832091812458, 4724

[5478.393402130357, 4826.704454970039, 4566.583304039659, 4426.915823952944, 4338.6652612212165, 4267.454065845393, 4221.47573232579, 4187.107870044545, 4156.39214175331, 4122.583626087349, 4091.709317318435, 4069.359608256848, 4056.3493343410228, 4048.35447222548, 4042.687544860523, 4037.987637676032, 4032.4716642789544, 4026.02964330702, 4019.1940735926096, 4013.0636231230183, 4006.639810358843, 4001.446450968814, 3997.745210195055, 3994.9733337612097, 3992.4083133934387, 3990.2348358055287, 3988.463050871342, 3986.8465225895798, 3985.499894250094, 3984.6196178946784, 3984.0026546474824, 3983.5218289289824, 3983.0624789126286, 3982.5548047078087, 3982.0194403188466, 3981.451101428808, 3980.833921481903, 3980.2085855502896, 3979.569435721995, 3978.9524841666807, 3978.218954279644, 3977.4335077132478, 3976.579412329891, 3975.757566129353, 3975.1754111444393, 3974.7264306462193, 3974.3749830399797, 3974.095836008538, 3973.9056937210735, 3973.7821517441944, 3973.7075120101017, 3973.65639

[5512.839660312679, 4797.350428320453, 4449.699929546325, 4298.908691193318, 4223.686763535216, 4176.362719706686, 4140.81407882928, 4115.02671829375, 4097.065180140981, 4083.222438818098, 4072.781220292666, 4064.797561218067, 4059.280902974756, 4054.6820239167814, 4050.917729849666, 4047.2091748601874, 4043.932749544795, 4040.870251996973, 4038.0514184827307, 4035.8106072249057, 4034.1713931144163, 4032.887739284534, 4031.878279395982, 4030.703490973904, 4029.851491529828, 4029.2523857462315, 4028.8139400988866, 4028.3788084410057, 4027.9584163877225, 4027.331777232494, 4025.833931729523, 4023.3298179086655, 4020.566804713644, 4018.2848817333625, 4016.2746894665797, 4014.254817746145, 4012.319272080981, 4011.3305012530413, 4010.7206857649253, 4010.266866548101, 4009.825062222065, 4009.352243497588, 4008.870926800074, 4008.4600085687366, 4008.0391719233203, 4007.698538930383, 4007.389116758158, 4007.044126358035, 4006.633382009464, 4006.268923309983, 4005.940830175279, 4005.71857889977

Total time taken to converge 15.83600902299986

Iterations total taken for convergence : 0


Cost variation over iterations
['7043.184']

Balance variation over iterations
['0.49353049907578556']

Fairness error over iterations
['6.851199767323856e-05']


Final converged balance : 0.49353049907578556

Final Converged Fairness Error : 6.851199767323856e-05

Final converged Clustering Objective/Cost 7043.184487196156

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
                        RUN  : 8
[5194.762473805224, 4622.162195414357, 4394.829126109085, 4275.572200967557, 4208.326433992973, 4167.682599913441, 4139.251431009525, 4119.714317471302, 4105.115604067052, 4092.868378503859, 4083.0822439963868, 4074.6781869510905, 4066.5112983679255, 4056.032603048036, 4046.705643454035, 4041.5733417816764, 4037.6728834848645, 4034.223320224829, 4030.4228739945715, 4026.5846106441154, 4022.0245679718905, 4017.78175860996, 4013.333616767474, 4